In [19]:
# Bloc 3.5 – Résumé M1 sur la plage centrale de k (k_core_min=10, k_core_max=18)

import pandas as pd
from pathlib import Path
import numpy as np

# 3.5.1 – Paramètres de la plage centrale
K_CORE_MIN = 10
K_CORE_MAX = 18

if K_CORE_MIN < 5 or K_CORE_MAX > 20:
    raise ValueError("La plage centrale doit rester dans [5, 20].")

if K_CORE_MIN >= K_CORE_MAX:
    raise ValueError("K_CORE_MIN doit être strictement inférieur à K_CORE_MAX.")

print(f"Plage centrale de travail pour M1 (S0) : k ∈ [{K_CORE_MIN}, {K_CORE_MAX}]")

# 3.5.2 – Chargement des résultats M1 par k
DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
PER_K_PATH = DEST_DIR / "M1_S0_calibration_per_k.csv"

if not PER_K_PATH.exists():
    raise FileNotFoundError(
        f"M1_S0_calibration_per_k.csv introuvable à : {PER_K_PATH}.\n"
        "Assurez-vous que le Bloc 3.3 a bien été exécuté."
    )

df_per_k = pd.read_csv(PER_K_PATH)

print("\nAperçu de M1_S0_calibration_per_k.csv :")
display(df_per_k.head())

# 3.5.3 – Filtre sur la plage centrale de k
df_core = df_per_k[
    (df_per_k["k"] >= K_CORE_MIN) & (df_per_k["k"] <= K_CORE_MAX)
].copy()

if df_core.empty:
    raise RuntimeError(
        f"Aucune ligne après filtrage k ∈ [{K_CORE_MIN}, {K_CORE_MAX}]."
    )

print(f"\nNombre de lignes après filtrage sur k ∈ [{K_CORE_MIN}, {K_CORE_MAX}] : {len(df_core)}")

# 3.5.4 – Résumé par combinaison (W, G) sur la plage centrale
group_cols = ["series", "window_size_months", "stride_months"]
records_core = []

for (series, W, G), group in df_core.groupby(group_cols, sort=True):
    d_vals = group["d_hat_k"].to_numpy(dtype=float)
    ks = group["k"].to_numpy(dtype=int)

    record = {
        "series": series,
        "window_size_months": int(W),
        "stride_months": int(G),
        "k_core_min": int(ks.min()),
        "k_core_max": int(ks.max()),
        "d_core_mean": float(d_vals.mean()),
        "d_core_std": float(d_vals.std()),
        "d_core_min": float(d_vals.min()),
        "d_core_max": float(d_vals.max()),
        "n_k_core": int(len(d_vals)),
    }
    records_core.append(record)

df_summary_core = pd.DataFrame.from_records(records_core)

print("\nRésumé M1 sur la plage centrale de k (par combinaison W,G) :")
display(df_summary_core)

# 3.5.5 – Sauvegarde des résumés
CORE_SUMMARY_PATH = DEST_DIR / "M1_S0_calibration_summary_core_k.csv"
df_summary_core.to_csv(CORE_SUMMARY_PATH, index=False)

print("\nFichier de résumé (plage centrale de k) sauvegardé :")
print(f"  - {CORE_SUMMARY_PATH}")

# 3.5.6 – Logging
log_message(
    "INFO",
    (
        f"Résumé M1 S0 sur plage centrale de k (k_core_min={K_CORE_MIN}, "
        f"k_core_max={K_CORE_MAX}) sauvegardé dans {CORE_SUMMARY_PATH.name}."
    ),
    block="BLOC_3.5",
)
log_metric(
    "M1_S0_core_k_summary_created",
    True,
    extra={
        "k_core_min": int(K_CORE_MIN),
        "k_core_max": int(K_CORE_MAX),
        "path": str(CORE_SUMMARY_PATH),
    },
)

Plage centrale de travail pour M1 (S0) : k ∈ [10, 18]

Aperçu de M1_S0_calibration_per_k.csv :


,series,window_size_months,stride_months,k,d_hat_k,n_windows
0,S0,132,12,5,6.611780,40
1,S0,132,12,6,5.703214,40
2,S0,132,12,7,5.265625,40
3,S0,132,12,8,4.916382,40
4,S0,132,12,9,4.520347,40



Nombre de lignes après filtrage sur k ∈ [10, 18] : 81

Résumé M1 sur la plage centrale de k (par combinaison W,G) :


,series,window_size_months,stride_months,k_core_min,k_core_max,d_core_mean,d_core_std,d_core_min,d_core_max,n_k_core
0,S0,60,1,10,18,3.301630,0.404120,2.801035,4.038216,9
1,S0,60,6,10,18,3.413963,0.388647,2.985436,4.125657,9
2,S0,60,12,10,18,3.394886,0.393295,2.920511,4.204442,9
3,S0,132,1,10,18,3.525000,0.209171,3.317934,3.954966,9
4,S0,132,6,10,18,3.603652,0.230090,3.333395,4.045191,9
5,S0,132,12,10,18,3.563428,0.305607,3.225729,4.193233,9
6,S0,264,1,10,18,4.611742,0.382151,4.111676,5.256736,9
7,S0,264,6,10,18,4.569296,0.287942,4.144299,5.134880,9
8,S0,264,12,10,18,4.704547,0.350419,4.221529,5.379095,9



Fichier de résumé (plage centrale de k) sauvegardé :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration\M1_S0_calibration_summary_core_k.csv
[STEP=17][INFO][BLOC_3.5] Résumé M1 S0 sur plage centrale de k (k_core_min=10, k_core_max=18) sauvegardé dans M1_S0_calibration_summary_core_k.csv.
[METRIC][M1_S0_core_k_summary_created] = True (step=17)


### Bloc 3.6 – Synthèse M1 (Levina–Bickel) sur la plage centrale de k pour S0

À partir du fichier  
`data_phase2/d_estimates_calibration/M1_S0_calibration_summary_core_k.csv`  
(et avec la plage centrale de travail `k_core_min = 10`, `k_core_max = 18`), nous obtenons
pour la série S0 (Sunspots brute nettoyée) :

| W (mois) | G (mois) | d_core_mean | d_core_std | d_core_min | d_core_max |
|----------|----------|-------------|------------|------------|------------|
| 60       | 1        | ≈ 3.30      | ≈ 0.40     | ≈ 2.80     | ≈ 4.04     |
| 60       | 6        | ≈ 3.41      | ≈ 0.39     | ≈ 2.99     | ≈ 4.13     |
| 60       | 12       | ≈ 3.39      | ≈ 0.39     | ≈ 2.92     | ≈ 4.20     |
| 132      | 1        | ≈ 3.53      | ≈ 0.21     | ≈ 3.32     | ≈ 3.95     |
| 132      | 6        | ≈ 3.60      | ≈ 0.23     | ≈ 3.33     | ≈ 4.05     |
| 132      | 12       | ≈ 3.56      | ≈ 0.31     | ≈ 3.23     | ≈ 4.19     |
| 264      | 1        | ≈ 4.61      | ≈ 0.38     | ≈ 4.11     | ≈ 5.26     |
| 264      | 6        | ≈ 4.57      | ≈ 0.29     | ≈ 4.14     | ≈ 5.13     |
| 264      | 12       | ≈ 4.70      | ≈ 0.35     | ≈ 4.22     | ≈ 5.38     |

Principaux constats :

1. **Influence de la taille de fenêtre W**
   - Pour **W = 60** et **W = 132**, `d_core_mean` est typiquement entre **3.3 et 3.6**.
   - Pour **W = 264**, `d_core_mean` monte vers **4.6–4.7**.
   → La taille de fenêtre a un effet important sur la valeur estimée de `d`.

2. **Effet limité du pas de glissement G**
   - Pour un W donné, les trois pas (**G = 1, 6, 12 mois**) donnent des `d_core_mean`
     très proches, avec des écarts relativement faibles.
   → M1 est peu sensible au choix de G (au moins sur ce sous-échantillon).

3. **Incertitude liée au choix de k (zone centrale)**
   - Les `d_core_std` sont de l’ordre de **0.2–0.4**.
   - Les intervalles `[d_core_min, d_core_max]` couvrent typiquement ~0.8–1 unité de `d`.
   → L’estimation de `d` n’est pas un nombre unique : il faut raisonner en termes
     de **plages plausibles** pour chaque (W, G).

4. **Lien avec T_log**
   - Pour W=60/132, les `d_core_mean` autour de **3.3–3.6** indiquent que
     \\( d - 4 \\) est souvent négatif ou proche de zéro sur ces échelles → T_log
     se situe près de la frontière entre régimes.
   - Pour W=264, \\( d \\approx 4.6 \\) suggère \\( d - 4 > 0 \\) sur des fenêtres plus longues
     → interprétation plus clairement “divergente” si l’on choisit ces fenêtres.

Ces résultats seront utilisés plus tard :

- pour comparer M1 avec d’autres méthodes (M2 PR/PCA, M3 spectrale) ;
- pour propager l’incertitude sur `d` vers `T_log(n, d)` ;
- pour analyser comment les choix (W, G, k) influencent les conclusions sur la dynamique.

### Bloc 3.7 – Méthode M1 sur S1 (z‑score) – Sous‑échantillon de calibration

Objectif : appliquer la même méthode M1 (Levina–Bickel) que pour S0, mais sur la série
normalisée **S1** (z‑score), afin de tester la **robustesse de `d` à un changement d’échelle**.

Rappel :

- S1 est définie à partir de S0 par une normalisation globale (z‑score) de  
  `Monthly Mean Total Sunspot Number`, et sauvegardée dans :  
  `data_phase2/sunspots_clean/Sunspots_clean_zscore.csv`.
- Nous utilisons le **même sous-échantillon de fenêtres de calibration**  
  (`window_calibration_subset.csv`) et la **même plage de k** que pour S0 :
  - plage complète de diagnostic : `k ∈ [5, 20]` (K_MIN, K_MAX)
  - plage centrale de travail : `k_core_min = 10`, `k_core_max = 18` (K_CORE_MIN, K_CORE_MAX)

Dans ce bloc, nous allons :

1. Recharger S1 depuis `Sunspots_clean_zscore.csv` et reconstruire les matrices de fenêtres
   de calibration S1 (une matrice par couple `(W, G)`), sauvegardées dans :  
   `data_phase2/windows/calibration_matrices/S1_W{W}_G{G}_calib.csv`.
2. Appliquer M1 (Levina–Bickel) à ces matrices :
   - calculer `d_hat(k)` pour `k = K_MIN..K_MAX` pour chaque (W,G) ;
   - sauvegarder les résultats détaillés par k dans :  
     `M1_S1_calibration_per_k.csv`  
     et les résumés globaux (tous k) dans :  
     `M1_S1_calibration_summary.csv`.
3. Calculer, sur la **plage centrale** `k ∈ [K_CORE_MIN, K_CORE_MAX]`, un résumé :  
   `d_core_mean`, `d_core_std`, `d_core_min`, `d_core_max`  
   dans : `M1_S1_calibration_summary_core_k.csv`.

Nous pourrons ensuite comparer directement S0 vs S1 pour voir si la dimension effective
estimée est stable ou fortement dépendante de l’échelle des valeurs.

In [20]:
# Bloc 3.7 – M1 Levina–Bickel sur S1 (z-score) – Sous-échantillon de calibration

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.neighbors import NearestNeighbors  # déjà importé mais sans risque

# 3.7.1 – Vérifications des paramètres et fonctions M1 déjà définis

try:
    K_MIN, K_MAX
except NameError:
    raise RuntimeError(
        "K_MIN/K_MAX ne sont pas définis. "
        "Assurez-vous d'avoir exécuté le Bloc 3.3 (M1 sur S0) avant ce bloc."
    )

try:
    K_CORE_MIN, K_CORE_MAX
except NameError:
    raise RuntimeError(
        "K_CORE_MIN/K_CORE_MAX ne sont pas définis. "
        "Assurez-vous d'avoir exécuté le Bloc 3.5 (plage centrale) avant ce bloc."
    )

try:
    levina_bickel_d_over_k_range
except NameError:
    raise RuntimeError(
        "La fonction levina_bickel_d_over_k_range n'est pas définie. "
        "Assurez-vous d'avoir exécuté le Bloc 3.3 (implémentation M1) avant ce bloc."
    )

print(f"Paramètres M1 : K_MIN={K_MIN}, K_MAX={K_MAX}, "
      f"K_CORE_MIN={K_CORE_MIN}, K_CORE_MAX={K_CORE_MAX}")

# 3.7.2 – Rechargement de la série S1 (z-score)

DATA_PHASE2_CLEAN_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_clean"
SUNSPOTS_ZSCORE_CSV_PATH = DATA_PHASE2_CLEAN_DIR / "Sunspots_clean_zscore.csv"

if not SUNSPOTS_ZSCORE_CSV_PATH.exists():
    raise FileNotFoundError(
        f"Sunspots_clean_zscore.csv introuvable à : {SUNSPOTS_ZSCORE_CSV_PATH}.\n"
        "Assurez-vous que le Bloc 2.3 a été exécuté."
    )

df_sunspots_z = pd.read_csv(SUNSPOTS_ZSCORE_CSV_PATH, parse_dates=["Date"])
df_ts_S1 = df_sunspots_z.sort_values("Date").set_index("Date")

value_col = "Monthly Mean Total Sunspot Number"

print("\nSérie S1 (z-score) rechargée :")
print(f"  - shape = {df_ts_S1.shape}")
print(f"  - index min/max = {df_ts_S1.index.min().date()} -> {df_ts_S1.index.max().date()}")

# 3.7.3 – Chargement du sous-échantillon de calibration

WINDOWS_DIR = PHASE2_ROOT / "data_phase2" / "windows"
WINDOW_CALIB_PATH = WINDOWS_DIR / "window_calibration_subset.csv"

if not WINDOW_CALIB_PATH.exists():
    raise FileNotFoundError(
        f"window_calibration_subset.csv introuvable à : {WINDOW_CALIB_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.1."
    )

df_windows_calib = pd.read_csv(
    WINDOW_CALIB_PATH,
    parse_dates=["start_date", "end_date"],
)

print(f"\nSous-échantillon de calibration chargé : {len(df_windows_calib)} fenêtres.")
print("Aperçu des 5 premières lignes :")
display(df_windows_calib.head())

# 3.7.4 – Construction des matrices S1 (z-score) pour chaque combinaison (W,G)

CALIB_MATRICES_DIR = WINDOWS_DIR / "calibration_matrices"
CALIB_MATRICES_DIR.mkdir(parents=True, exist_ok=True)

groups = df_windows_calib.groupby(["window_size_months", "stride_months"], sort=True)

matrix_files_S1 = []

for (W, G), group in groups:
    group = group.sort_values("start_index").reset_index(drop=True)
    window_ids = group["window_id"].astype(int).to_numpy()
    n_windows = len(group)

    rows = []
    for _, row in group.iterrows():
        start_idx = int(row["start_index"])
        end_idx = int(row["end_index"])
        values = df_ts_S1.iloc[start_idx : end_idx + 1][value_col].to_numpy()

        if len(values) != W:
            raise RuntimeError(
                f"S1: longueur inattendue pour window_id={row['window_id']} "
                f"(attendu W={W}, obtenu {len(values)})."
            )

        rows.append(values)

    X = np.vstack(rows)

    col_names = ["window_id"] + [f"t_{i}" for i in range(W)]
    data = np.column_stack([window_ids, X])
    df_matrix_S1 = pd.DataFrame(data, columns=col_names)

    out_name = f"S1_W{W}_G{G}_calib.csv"
    out_path = CALIB_MATRICES_DIR / out_name
    df_matrix_S1.to_csv(out_path, index=False)

    print(
        f"\nMatrice de calibration S1 construite pour (W={W}, G={G}) : "
        f"{df_matrix_S1.shape[0]} fenêtres x {W} colonnes de valeurs."
    )
    print(f"  - Fichier sauvegardé : {out_path}")

    matrix_files_S1.append(out_path)

# 3.7.5 – Application de M1 (Levina–Bickel) sur les matrices S1

records_per_k_S1 = []
records_summary_S1 = []
records_summary_core_S1 = []

for matrix_path in sorted(matrix_files_S1):
    name = matrix_path.name  # ex: S1_W60_G1_calib.csv
    base = name.replace("S1_W", "").replace("_calib.csv", "")
    W_str, G_part = base.split("_G")
    W = int(W_str)
    G = int(G_part)

    df_matrix = pd.read_csv(matrix_path)
    X = df_matrix.drop(columns=["window_id"]).to_numpy()
    n_windows, W_check = X.shape
    assert W_check == W, f"Incohérence S1: W={W} mais X.shape[1]={W_check}"

    print(f"\nM1 sur {name} : n_windows={n_windows}, W={W}")

    # d_hat(k) pour k=K_MIN..K_MAX (borné par n_windows-1)
    k_max_eff = min(K_MAX, n_windows - 1)
    d_by_k = levina_bickel_d_over_k_range(X, K_MIN, k_max_eff)

    # enregistrement global par k
    for k, d_hat_k in d_by_k.items():
        records_per_k_S1.append(
            {
                "series": "S1",
                "window_size_months": W,
                "stride_months": G,
                "k": k,
                "d_hat_k": d_hat_k,
                "n_windows": n_windows,
            }
        )

    # résumé global sur k ∈ [K_MIN, k_max_eff]
    d_vals_all = np.array(list(d_by_k.values()), dtype=float)
    summary_all = {
        "series": "S1",
        "window_size_months": W,
        "stride_months": G,
        "k_min": min(d_by_k.keys()),
        "k_max": max(d_by_k.keys()),
        "d_mean": float(d_vals_all.mean()),
        "d_std": float(d_vals_all.std()),
        "d_min": float(d_vals_all.min()),
        "d_max": float(d_vals_all.max()),
        "n_windows": n_windows,
    }
    records_summary_S1.append(summary_all)

    # résumé restreint à la plage centrale [K_CORE_MIN, K_CORE_MAX]
    d_by_k_core = {
        k: v
        for k, v in d_by_k.items()
        if K_CORE_MIN <= k <= K_CORE_MAX
    }
    d_vals_core = np.array(list(d_by_k_core.values()), dtype=float)
    summary_core = {
        "series": "S1",
        "window_size_months": W,
        "stride_months": G,
        "k_core_min": min(d_by_k_core.keys()),
        "k_core_max": max(d_by_k_core.keys()),
        "d_core_mean": float(d_vals_core.mean()),
        "d_core_std": float(d_vals_core.std()),
        "d_core_min": float(d_vals_core.min()),
        "d_core_max": float(d_vals_core.max()),
        "n_k_core": len(d_vals_core),
        "n_windows": n_windows,
    }
    records_summary_core_S1.append(summary_core)

    print(
        f"  -> S1 (tous k) : d_mean={summary_all['d_mean']:.3f}, d_std={summary_all['d_std']:.3f}"
    )
    print(
        f"  -> S1 (k∈[{summary_core['k_core_min']},{summary_core['k_core_max']}]) : "
        f"d_core_mean={summary_core['d_core_mean']:.3f}, "
        f"d_core_std={summary_core['d_core_std']:.3f}"
    )

# 3.7.6 – DataFrames et sauvegarde des résultats S1

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
DEST_DIR.mkdir(parents=True, exist_ok=True)

df_per_k_S1 = pd.DataFrame.from_records(records_per_k_S1)
df_summary_S1 = pd.DataFrame.from_records(records_summary_S1)
df_summary_core_S1 = pd.DataFrame.from_records(records_summary_core_S1)

PER_K_S1_PATH = DEST_DIR / "M1_S1_calibration_per_k.csv"
SUMMARY_S1_PATH = DEST_DIR / "M1_S1_calibration_summary.csv"
SUMMARY_CORE_S1_PATH = DEST_DIR / "M1_S1_calibration_summary_core_k.csv"

df_per_k_S1.to_csv(PER_K_S1_PATH, index=False)
df_summary_S1.to_csv(SUMMARY_S1_PATH, index=False)
df_summary_core_S1.to_csv(SUMMARY_CORE_S1_PATH, index=False)

print("\nFichiers de résultats M1 (S1, calibration) sauvegardés :")
print(f"  - {PER_K_S1_PATH}")
print(f"  - {SUMMARY_S1_PATH}")
print(f"  - {SUMMARY_CORE_S1_PATH}")

print("\nRésumé M1 S1 (plage centrale de k) :")
display(df_summary_core_S1)

# 3.7.7 – Logging

log_message(
    "INFO",
    (
        "M1 Levina–Bickel appliqué à S1 (z-score) sur le sous-échantillon de calibration "
        f"pour {len(records_summary_core_S1)} combinaisons (W,G). "
        f"Résultats sauvegardés dans {DEST_DIR}."
    ),
    block="BLOC_3.7",
)
log_metric(
    "M1_S1_calibration_combos",
    int(len(records_summary_core_S1)),
    extra={
        "per_k_path": str(PER_K_S1_PATH),
        "summary_path": str(SUMMARY_S1_PATH),
        "summary_core_path": str(SUMMARY_CORE_S1_PATH),
    },
)

Paramètres M1 : K_MIN=5, K_MAX=20, K_CORE_MIN=10, K_CORE_MAX=18

Série S1 (z-score) rechargée :
  - shape = (3265, 1)
  - index min/max = 1749-01-31 -> 2021-01-31

Sous-échantillon de calibration chargé : 360 fenêtres.
Aperçu des 5 premières lignes :


,window_id,window_size_months,stride_months,start_index,end_index,start_date,end_date,n_points
0,0,60,1,0,59,1749-01-31,1753-12-31,60
1,82,60,1,82,141,1755-11-30,1760-10-31,60
2,164,60,1,164,223,1762-09-30,1767-08-31,60
3,246,60,1,246,305,1769-07-31,1774-06-30,60
4,328,60,1,328,387,1776-05-31,1781-04-30,60



Matrice de calibration S1 construite pour (W=60, G=1) : 40 fenêtres x 60 colonnes de valeurs.
  - Fichier sauvegardé : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\windows\calibration_matrices\S1_W60_G1_calib.csv

Matrice de calibration S1 construite pour (W=60, G=6) : 40 fenêtres x 60 colonnes de valeurs.
  - Fichier sauvegardé : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\windows\calibration_matrices\S1_W60_G6_calib.csv

Matrice de calibration S1 construite pour (W=60, G=12) : 40 fenêtres x 60 colonnes de valeurs.
  - Fichier sauvegardé : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\windows\calibration_matrices\S1_W60_G12_calib.csv

Matrice de calibration S1 construite pour (W=132, G=1) : 40 fenêtres x 132 colonnes de valeurs.
  - Fichier sauvegardé : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\windows\calibration_matrices\S1_W132_G1_calib.csv

Ma

,series,window_size_months,stride_months,k_core_min,k_core_max,d_core_mean,d_core_std,d_core_min,d_core_max,n_k_core,n_windows
0,S1,132,12,10,18,3.563428,0.305607,3.225729,4.193233,9,40
1,S1,132,1,10,18,3.525000,0.209171,3.317934,3.954966,9,40
2,S1,132,6,10,18,3.603652,0.230090,3.333395,4.045191,9,40
3,S1,264,12,10,18,4.704547,0.350419,4.221529,5.379095,9,40
4,S1,264,1,10,18,4.611742,0.382151,4.111676,5.256736,9,40
5,S1,264,6,10,18,4.569296,0.287942,4.144299,5.134880,9,40
6,S1,60,12,10,18,3.394886,0.393295,2.920511,4.204442,9,40
7,S1,60,1,10,18,3.301630,0.404120,2.801035,4.038216,9,40
8,S1,60,6,10,18,3.413963,0.388647,2.985436,4.125657,9,40


[STEP=18][INFO][BLOC_3.7] M1 Levina–Bickel appliqué à S1 (z-score) sur le sous-échantillon de calibration pour 9 combinaisons (W,G). Résultats sauvegardés dans C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration.
[METRIC][M1_S1_calibration_combos] = 9 (step=18)


### Bloc 3.8 – Méthode M2 : Participation Ratio / PCA sur le sous‑échantillon (S0)

Objectif : estimer une dimension effective `d_PR` basée sur l’analyse en composantes principales
(PCA) / Participation Ratio, en utilisant le **même sous-échantillon de fenêtres** que pour M1,
et les matrices S0 construites en Bloc 3.2 :

- `data_phase2/windows/calibration_matrices/S0_W{W}_G{G}_calib.csv`

Pour chaque combinaison `(W, G)` :

1. On lit la matrice `X` de taille `(n_windows, W)` (40 fenêtres × W mois).
2. On centre les colonnes (soustraction de la moyenne sur les fenêtres).
3. On calcule la matrice de covariance (sur les colonnes) et ses valeurs propres \\( \lambda_i \\).
4. On en déduit :

   - la **Participation Ratio dimension** :

     \[
     d_{\mathrm{PR}} = \frac{\left(\sum_i \lambda_i\right)^2}{\sum_i \lambda_i^2}
     = \frac{1}{\sum_i p_i^2}, \quad p_i = \frac{\lambda_i}{\sum_j \lambda_j}
     \]

   - la **dimension à 90 % de variance expliquée** : plus petit `m` tel que  
     \\( \sum_{i=1}^m p_i \ge 0.9 \\) (optionnel, pour diagnostic).

Les résultats seront sauvegardés dans :

- `data_phase2/d_estimates_calibration/M2_S0_PR_calibration_summary.csv`

avec, pour chaque `(W, G)` :

- `d_PR`, `d_PR_80`, `d_PR_90` (Participation Ratio + dimensions à 80 % et 90 % de variance),
- nombre de fenêtres, nombre de dimensions significatives, etc.

Ce bloc permet de comparer M2 avec M1 (Levina–Bickel) sur exactement les mêmes configurations,
avant d’étendre les estimations de `d` à toutes les fenêtres.

In [21]:
# Bloc 3.8 – M2 Participation Ratio / PCA sur S0 – Sous-échantillon de calibration

import numpy as np
import pandas as pd
from pathlib import Path

# 3.8.1 – Dossier des matrices de calibration S0
CALIB_MATRICES_DIR = PHASE2_ROOT / "data_phase2" / "windows" / "calibration_matrices"

if not CALIB_MATRICES_DIR.exists():
    raise FileNotFoundError(
        f"Le dossier des matrices de calibration n'existe pas : {CALIB_MATRICES_DIR}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.2."
    )

matrix_files_S0 = sorted(
    [p for p in CALIB_MATRICES_DIR.glob("S0_W*_G*_calib.csv") if p.is_file()]
)

if not matrix_files_S0:
    raise RuntimeError(
        f"Aucune matrice S0_W*_G*_calib.csv trouvée dans {CALIB_MATRICES_DIR}."
    )

print("Fichiers de matrices S0 de calibration trouvés pour M2 :")
for p in matrix_files_S0:
    print(f"  - {p.name}")

# 3.8.2 – Fonction utilitaire : calcul de la dimension PR et des dimensions à 80/90%

def pr_dimensions_from_cov_eigvals(eigvals, var_thresholds=(0.8, 0.9)):
    """
    eigvals : array des valeurs propres (non négatives), triées décroissantes.
    var_thresholds : seuils de variance expliquée (ex: (0.8, 0.9)).

    Retourne :
      d_PR, dict {seuil -> d_seuil}
    """
    # Filtrer les valeurs propres non positives (sécurité numérique)
    eigvals = np.asarray(eigvals, dtype=float)
    eigvals = eigvals[eigvals > 0]

    if eigvals.size == 0:
        return 0.0, {thr: 0 for thr in var_thresholds}

    total = eigvals.sum()
    p = eigvals / total

    # Participation Ratio
    d_PR = 1.0 / np.sum(p**2)

    # Dimensions à seuil de variance expliquée
    cum_p = np.cumsum(p)
    d_by_thr = {}
    for thr in var_thresholds:
        idx = np.searchsorted(cum_p, thr, side="left")
        d_by_thr[thr] = int(idx + 1)  # +1 car idx est un index 0-based
    return float(d_PR), d_by_thr

# 3.8.3 – Application de M2 sur toutes les matrices S0 de calibration

records_M2_S0 = []

for matrix_path in matrix_files_S0:
    name = matrix_path.name  # ex: S0_W60_G1_calib.csv
    base = name.replace("S0_W", "").replace("_calib.csv", "")
    W_str, G_part = base.split("_G")
    W = int(W_str)
    G = int(G_part)

    df_matrix = pd.read_csv(matrix_path)
    X = df_matrix.drop(columns=["window_id"]).to_numpy(dtype=float)

    n_windows, W_check = X.shape
    assert W_check == W, f"Incohérence S0: W={W} mais X.shape[1]={W_check}"

    # Centrage des colonnes (chaque position temporelle a moyenne 0 sur les fenêtres)
    X_centered = X - X.mean(axis=0, keepdims=True)

    # Covariance (colonnes = dimensions)
    # rowvar=False -> variables en colonnes
    cov = np.cov(X_centered, rowvar=False)

    # Valeurs propres (symétrique) -> eigvalsh
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.flip(np.sort(eigvals))  # décroissant

    d_PR, d_by_thr = pr_dimensions_from_cov_eigvals(eigvals, var_thresholds=(0.8, 0.9))
    d_80 = d_by_thr[0.8]
    d_90 = d_by_thr[0.9]

    record = {
        "series": "S0",
        "window_size_months": W,
        "stride_months": G,
        "n_windows": n_windows,
        "W": W,
        "d_PR": d_PR,
        "d_PR_80": d_80,
        "d_PR_90": d_90,
        "n_eigvals_pos": int((eigvals > 0).sum()),
    }
    records_M2_S0.append(record)

    print(
        f"\nM2 PR/PCA sur {name} : n_windows={n_windows}, W={W}, "
        f"d_PR={d_PR:.3f}, d_PR_80={d_80}, d_PR_90={d_90}"
    )

# 3.8.4 – DataFrame de résultats et sauvegarde

df_M2_S0 = pd.DataFrame.from_records(records_M2_S0)

print("\nRésumé M2 (S0, sous-échantillon) :")
display(df_M2_S0)

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
DEST_DIR.mkdir(parents=True, exist_ok=True)

M2_S0_SUMMARY_PATH = DEST_DIR / "M2_S0_PR_calibration_summary.csv"
df_M2_S0.to_csv(M2_S0_SUMMARY_PATH, index=False)

print("\nFichier de résultats M2 (S0, PR/PCA, calibration) sauvegardé :")
print(f"  - {M2_S0_SUMMARY_PATH}")

# 3.8.5 – Logging

log_message(
    "INFO",
    (
        "M2 PR/PCA appliqué à S0 sur le sous-échantillon de calibration "
        f"pour {len(records_M2_S0)} combinaisons (W,G). "
        f"Résultats sauvegardés dans {M2_S0_SUMMARY_PATH.name}."
    ),
    block="BLOC_3.8",
)
log_metric(
    "M2_S0_calibration_combos",
    int(len(records_M2_S0)),
    extra={
        "summary_path": str(M2_S0_SUMMARY_PATH),
    },
)

Fichiers de matrices S0 de calibration trouvés pour M2 :
  - S0_W132_G12_calib.csv
  - S0_W132_G1_calib.csv
  - S0_W132_G6_calib.csv
  - S0_W264_G12_calib.csv
  - S0_W264_G1_calib.csv
  - S0_W264_G6_calib.csv
  - S0_W60_G12_calib.csv
  - S0_W60_G1_calib.csv
  - S0_W60_G6_calib.csv

M2 PR/PCA sur S0_W132_G12_calib.csv : n_windows=40, W=132, d_PR=4.022, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S0_W132_G1_calib.csv : n_windows=40, W=132, d_PR=4.063, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S0_W132_G6_calib.csv : n_windows=40, W=132, d_PR=4.051, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S0_W264_G12_calib.csv : n_windows=40, W=264, d_PR=5.006, d_PR_80=5, d_PR_90=10

M2 PR/PCA sur S0_W264_G1_calib.csv : n_windows=40, W=264, d_PR=5.046, d_PR_80=5, d_PR_90=10

M2 PR/PCA sur S0_W264_G6_calib.csv : n_windows=40, W=264, d_PR=5.043, d_PR_80=5, d_PR_90=10

M2 PR/PCA sur S0_W60_G12_calib.csv : n_windows=40, W=60, d_PR=2.827, d_PR_80=2, d_PR_90=4

M2 PR/PCA sur S0_W60_G1_calib.csv : n_windows=40, W=60, d_PR=2.798, d

,series,window_size_months,stride_months,n_windows,W,d_PR,d_PR_80,d_PR_90,n_eigvals_pos
0,S0,132,12,40,132,4.021994,3,6,84
1,S0,132,1,40,132,4.063424,3,6,86
2,S0,132,6,40,132,4.050841,3,6,85
3,S0,264,12,40,264,5.006334,5,10,151
4,S0,264,1,40,264,5.046302,5,10,150
5,S0,264,6,40,264,5.043250,5,10,154
6,S0,60,12,40,60,2.826623,2,4,49
7,S0,60,1,40,60,2.798131,2,4,51
8,S0,60,6,40,60,2.839542,2,4,50



Fichier de résultats M2 (S0, PR/PCA, calibration) sauvegardé :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration\M2_S0_PR_calibration_summary.csv
[STEP=19][INFO][BLOC_3.8] M2 PR/PCA appliqué à S0 sur le sous-échantillon de calibration pour 9 combinaisons (W,G). Résultats sauvegardés dans M2_S0_PR_calibration_summary.csv.
[METRIC][M2_S0_calibration_combos] = 9 (step=19)


### Bloc 3.9 – Synthèse M1 vs M2 sur le sous‑échantillon (S0)

Sur le sous‑échantillon de calibration (360 fenêtres, S0) :

- **M1 (Levina–Bickel, plage centrale k ∈ [10,18])**  
  - W = 60 mois : `d_core_mean` ≈ **3.3–3.4**  
  - W = 132 mois : `d_core_mean` ≈ **3.5–3.6**  
  - W = 264 mois : `d_core_mean` ≈ **4.6–4.7**

- **M2 (PR/PCA)**  
  - W = 60 mois : `d_PR` ≈ **2.8–2.84**, `d_PR_90` = 4  
  - W = 132 mois : `d_PR` ≈ **4.02–4.06**, `d_PR_90` = 6  
  - W = 264 mois : `d_PR` ≈ **5.00–5.05**, `d_PR_90` = 10

Points communs :

1. **Effet de la taille de fenêtre W**

   - Les deux méthodes montrent une **augmentation de la dimension** quand W passe de 60 → 132 → 264 mois.  
   - Les ordres de grandeur sont compatibles :
     - petites fenêtres (5 ans) : d ≈ 3 (M1) vs ≈ 2.8 (M2) ;
     - fenêtres de cycle (~11 ans) : d ≈ 3.5–3.6 (M1) vs ≈ 4.0 (M2) ;
     - grandes fenêtres (~22 ans) : d ≈ 4.6–4.7 (M1) vs ≈ 5.0 (M2).

2. **Effet du pas de glissement G**

   - Pour un W donné, les valeurs sont très proches pour G = 1, 6, 12.  
   - M1 et M2 sont donc **faiblement sensibles à G** sur ce sous‑ensemble.

3. **Structure interne (PCA)**

   - Pour W = 60 : 2 composantes suffisent pour ~80 % de la variance, 4 pour ~90 %.  
   - Pour W = 132 : 3 composantes pour ~80 %, 6 pour ~90 %.  
   - Pour W = 264 : 5 composantes pour ~80 %, 10 pour ~90 %.  
   → La Participation Ratio `d_PR` est cohérente avec ces nombres de composantes.

Conclusion provisoire :

- M1 et M2 donnent une **image cohérente** de la complexité :
  - d autour de 3 pour W=60, proche de 4 pour W=132, autour de 5 pour W=264.  
- Les différences numériques (M1 plus bas à W=60, M2 un peu plus haut à W=264) reflètent
  les hypothèses différentes des méthodes (voisinages locaux vs variance globale),
  mais ne changent pas la **tendance qualitative**.

La prochaine étape consiste à vérifier que M2 est, comme M1, **invariante à la normalisation**
en appliquant PR/PCA à la série S1 (z‑score) sur le même sous‑échantillon.

### Bloc 3.10 – M2 (PR/PCA) sur S1 (z‑score) – Sous‑échantillon de calibration

Objectif : appliquer la méthode M2 (PR/PCA) à la série normalisée S1 (z‑score), en utilisant
les matrices de calibration S1 déjà construites en Bloc 3.7 :

- `data_phase2/windows/calibration_matrices/S1_W{W}_G{G}_calib.csv`

Nous allons :

1. lire chaque matrice S1 (`S1_W{W}_G{G}_calib.csv`) pour les 9 combinaisons (W,G) ;
2. centrer les colonnes (comme pour S0) ;
3. calculer les valeurs propres de la covariance, puis :
   - la dimension PR `d_PR`,
   - les dimensions `d_PR_80`, `d_PR_90` (80 % et 90 % de variance expliquée) ;
4. sauvegarder les résultats dans :

   - `data_phase2/d_estimates_calibration/M2_S1_PR_calibration_summary.csv`

Puis nous comparerons M2(S1) à M2(S0) pour vérifier la robustesse à la normalisation globale.

In [22]:
# Bloc 3.10 – M2 PR/PCA sur S1 (z-score) – Sous-échantillon de calibration

import numpy as np
import pandas as pd
from pathlib import Path

# 3.10.1 – Vérifier que la fonction PR existe (définie en 3.8)

try:
    pr_dimensions_from_cov_eigvals
except NameError:
    raise RuntimeError(
        "La fonction pr_dimensions_from_cov_eigvals n'est pas définie. "
        "Assurez-vous d'avoir exécuté le Bloc 3.8 (M2 sur S0) avant ce bloc."
    )

# 3.10.2 – Dossier des matrices de calibration S1

CALIB_MATRICES_DIR = PHASE2_ROOT / "data_phase2" / "windows" / "calibration_matrices"

if not CALIB_MATRICES_DIR.exists():
    raise FileNotFoundError(
        f"Le dossier des matrices de calibration n'existe pas : {CALIB_MATRICES_DIR}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.7."
    )

matrix_files_S1 = sorted(
    [p for p in CALIB_MATRICES_DIR.glob("S1_W*_G*_calib.csv") if p.is_file()]
)

if not matrix_files_S1:
    raise RuntimeError(
        f"Aucune matrice S1_W*_G*_calib.csv trouvée dans {CALIB_MATRICES_DIR}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.7."
    )

print("Fichiers de matrices S1 de calibration trouvés pour M2 :")
for p in matrix_files_S1:
    print(f"  - {p.name}")

# 3.10.3 – Application de M2 sur toutes les matrices S1 de calibration

records_M2_S1 = []

for matrix_path in matrix_files_S1:
    name = matrix_path.name  # ex: S1_W60_G1_calib.csv
    base = name.replace("S1_W", "").replace("_calib.csv", "")
    W_str, G_part = base.split("_G")
    W = int(W_str)
    G = int(G_part)

    df_matrix = pd.read_csv(matrix_path)
    X = df_matrix.drop(columns=["window_id"]).to_numpy(dtype=float)

    n_windows, W_check = X.shape
    assert W_check == W, f"Incohérence S1: W={W} mais X.shape[1]={W_check}"

    # Centrage des colonnes (chaque position temporelle a moyenne 0 sur les fenêtres)
    X_centered = X - X.mean(axis=0, keepdims=True)

    # Covariance (colonnes = dimensions)
    cov = np.cov(X_centered, rowvar=False)

    # Valeurs propres (symétrique)
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.flip(np.sort(eigvals))  # décroissant

    d_PR, d_by_thr = pr_dimensions_from_cov_eigvals(eigvals, var_thresholds=(0.8, 0.9))
    d_80 = d_by_thr[0.8]
    d_90 = d_by_thr[0.9]

    record = {
        "series": "S1",
        "window_size_months": W,
        "stride_months": G,
        "n_windows": n_windows,
        "W": W,
        "d_PR": d_PR,
        "d_PR_80": d_80,
        "d_PR_90": d_90,
        "n_eigvals_pos": int((eigvals > 0).sum()),
    }
    records_M2_S1.append(record)

    print(
        f"\nM2 PR/PCA sur {name} : n_windows={n_windows}, W={W}, "
        f"d_PR={d_PR:.3f}, d_PR_80={d_80}, d_PR_90={d_90}"
    )

# 3.10.4 – DataFrame de résultats et sauvegarde

df_M2_S1 = pd.DataFrame.from_records(records_M2_S1)

print("\nRésumé M2 (S1, sous-échantillon) :")
display(df_M2_S1)

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_calibration"
DEST_DIR.mkdir(parents=True, exist_ok=True)

M2_S1_SUMMARY_PATH = DEST_DIR / "M2_S1_PR_calibration_summary.csv"
df_M2_S1.to_csv(M2_S1_SUMMARY_PATH, index=False)

print("\nFichier de résultats M2 (S1, PR/PCA, calibration) sauvegardé :")
print(f"  - {M2_S1_SUMMARY_PATH}")

# 3.10.5 – Logging

log_message(
    "INFO",
    (
        "M2 PR/PCA appliqué à S1 (z-score) sur le sous-échantillon de calibration "
        f"pour {len(records_M2_S1)} combinaisons (W,G). "
        f"Résultats sauvegardés dans {M2_S1_SUMMARY_PATH.name}."
    ),
    block="BLOC_3.10",
)
log_metric(
    "M2_S1_calibration_combos",
    int(len(records_M2_S1)),
    extra={
        "summary_path": str(M2_S1_SUMMARY_PATH),
    },
)

Fichiers de matrices S1 de calibration trouvés pour M2 :
  - S1_W132_G12_calib.csv
  - S1_W132_G1_calib.csv
  - S1_W132_G6_calib.csv
  - S1_W264_G12_calib.csv
  - S1_W264_G1_calib.csv
  - S1_W264_G6_calib.csv
  - S1_W60_G12_calib.csv
  - S1_W60_G1_calib.csv
  - S1_W60_G6_calib.csv

M2 PR/PCA sur S1_W132_G12_calib.csv : n_windows=40, W=132, d_PR=4.022, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S1_W132_G1_calib.csv : n_windows=40, W=132, d_PR=4.063, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S1_W132_G6_calib.csv : n_windows=40, W=132, d_PR=4.051, d_PR_80=3, d_PR_90=6

M2 PR/PCA sur S1_W264_G12_calib.csv : n_windows=40, W=264, d_PR=5.006, d_PR_80=5, d_PR_90=10



M2 PR/PCA sur S1_W264_G1_calib.csv : n_windows=40, W=264, d_PR=5.046, d_PR_80=5, d_PR_90=10

M2 PR/PCA sur S1_W264_G6_calib.csv : n_windows=40, W=264, d_PR=5.043, d_PR_80=5, d_PR_90=10

M2 PR/PCA sur S1_W60_G12_calib.csv : n_windows=40, W=60, d_PR=2.827, d_PR_80=2, d_PR_90=4

M2 PR/PCA sur S1_W60_G1_calib.csv : n_windows=40, W=60, d_PR=2.798, d_PR_80=2, d_PR_90=4

M2 PR/PCA sur S1_W60_G6_calib.csv : n_windows=40, W=60, d_PR=2.840, d_PR_80=2, d_PR_90=4

Résumé M2 (S1, sous-échantillon) :


,series,window_size_months,stride_months,n_windows,W,d_PR,d_PR_80,d_PR_90,n_eigvals_pos
0,S1,132,12,40,132,4.021994,3,6,86
1,S1,132,1,40,132,4.063424,3,6,84
2,S1,132,6,40,132,4.050841,3,6,87
3,S1,264,12,40,264,5.006334,5,10,152
4,S1,264,1,40,264,5.046302,5,10,151
5,S1,264,6,40,264,5.043250,5,10,151
6,S1,60,12,40,60,2.826623,2,4,50
7,S1,60,1,40,60,2.798131,2,4,49
8,S1,60,6,40,60,2.839542,2,4,50



Fichier de résultats M2 (S1, PR/PCA, calibration) sauvegardé :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_calibration\M2_S1_PR_calibration_summary.csv
[STEP=20][INFO][BLOC_3.10] M2 PR/PCA appliqué à S1 (z-score) sur le sous-échantillon de calibration pour 9 combinaisons (W,G). Résultats sauvegardés dans M2_S1_PR_calibration_summary.csv.
[METRIC][M2_S1_calibration_combos] = 9 (step=20)


### Bloc 3.11 – Synthèse M2 (PR/PCA) sur S0 vs S1 (z‑score)

Nous avons appliqué M2 (PR/PCA) sur :

- **S0** : série brute nettoyée ([Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0))
- **S1** : série normalisée (z‑score, `Sunspots_clean_zscore.csv`)

en utilisant exactement les mêmes matrices de calibration (W, G, fenêtres).

#### Résultats principaux (d_PR, d_PR_80, d_PR_90)

Pour chaque combinaison `(W, G)`, les résultats S0 et S1 sont numériquement **quasi identiques** :

- **W = 60 mois**  
  - S0 : `d_PR ≈ 2.80–2.84`, `d_PR_80 = 2`, `d_PR_90 = 4`  
  - S1 : `d_PR ≈ 2.80–2.84`, `d_PR_80 = 2`, `d_PR_90 = 4`
- **W = 132 mois**  
  - S0 : `d_PR ≈ 4.02–4.06`, `d_PR_80 = 3`, `d_PR_90 = 6`  
  - S1 : `d_PR ≈ 4.02–4.06`, `d_PR_80 = 3`, `d_PR_90 = 6`
- **W = 264 mois**  
  - S0 : `d_PR ≈ 5.00–5.05`, `d_PR_80 = 5`, `d_PR_90 = 10`  
  - S1 : `d_PR ≈ 5.00–5.05`, `d_PR_80 = 5`, `d_PR_90 = 10`

#### Interprétation

1. **Invariance à la normalisation globale**

   - M2 donne la **même dimension effective** sur S0 et S1 pour toutes les tailles de fenêtre W
     et pas de glissement G considérés.
   - Donc, pour PR/PCA, un changement d’échelle global (z‑score) **n’affecte pas** les estimations
     de dimension : c’est cohérent avec la théorie (la covariance est invariante à un facteur
     d’échelle uniforme par fenêtre).

2. **Cohérence avec M1 (Levina–Bickel)**

   - Pour W=60 : M1 ≈ 3.3–3.4, M2 ≈ 2.8 → même ordre de grandeur, dimension modérée.
   - Pour W=132 : M1 ≈ 3.5–3.6, M2 ≈ 4.0 → très proche du seuil `d=4`, cohérent avec l’idée de
     “frontière critique”.
   - Pour W=264 : M1 ≈ 4.6–4.7, M2 ≈ 5.0 → dimension clairement > 4 sur des fenêtres longues.

   → Les deux méthodes montrent la même **structure qualitative** :  
   d augmente avec W, est peu sensible à G, et reste dans la plage ~3–5.

3. **Conséquence pour la Phase 2**

   - Nous avons maintenant **deux familles d’estimateurs de d (M1, M2) cohérents entre eux**  
     et **robustes à la normalisation globale** (S0 vs S1).
   - Les sources de variabilité les plus importantes sont maintenant :
     - la **taille de fenêtre W**,
     - la **méthode de dimension** (M1 vs M2),
     - la **plage de k** pour M1 (déjà analysée).

Ces résultats renforcent l’idée que la question centrale n’est pas l’unité ou l’échelle
des données, mais bien **le choix de l’échelle temporelle (fenêtre W) et de la méthode
d’estimation de d**.